# COMP551 Mini Project 2 - IMDB Sentiment Analysis  
This is the codes of mini project2 - IMDB Sentiment Analysis.  

## AUTHORS
Pengnan Fan, ID#260768510  

## TASKS
1. **Bernoulli Naive Bayes** (w/o any external library).  
2. **At least 2** out of 3 classifiers from the SciKit. i.e. suggestions: logistic regression, decision tree, or support vector machines  
3. **At least 2** different features extraction pipelines for processing the data.  
4. A model validation. i.e. **K-fold cross validation**  

## UPDATES
**<January 6, 2019>** Pengnan Fan create this notebook and implements functions readTrainData and readTestData  
> * **readTrainData(address:String):DataFrame**  
> This function takes a string **address** which indicates the address of your train data and will load comments and isPositive to a DataFrame.
> * **readTestData(address:String):DataFrame**  
> This function takes a string **address** which indicates the address of your test data and will load comments and isPositive to a DataFrame. **Note: all isPositive is initialized as 0**  
> * **Learning set** 25000 in total
> * **Test set** 25000 in total  

**<January 7, 2019>** Pengnan Fan implements wordsFrequencyNaive and wordsFrequencyStopword
> * **wordsFrequencyNaive(dataSet:DataFrame)**  
> This function takes a DataFrame **dataSet** and calculate the naive word frequency  
> * **wordsFrequencyStopword(dataSet:DataFrame)**  
> This function takes a DataFrame **dataSet** and calculate the word frequency without stopwords  

**<January 9, 2019>** Pengnan Fan uses sklearn.datasets.load_files(address:str) to load all datas. It helps to increase the speed of loading. And the implementation of Bernoulli naive Bayes has been started.  
> * **sklearn.datasets.load_files** - [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html)

**<January 10, 2019>** Pengnan Fan implements numOfExistanceNaive and numOfExistanceStopword
> * **numOfExistanceNaive(dataset:Bunch)**  
> This function takes a Bunch **dataSet** and calculate the naive existance of words  
> * **numOfExistanceStopword(dataset:Bunch)**  
> This funciton takes a Bunch **dataSet** and calculate the existance of words without stopwords  

In [1]:
import pandas as pd
import glob
import sys
from time import sleep
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import numpy
import scipy
import sklearn.datasets

## Task 0 - Preprocessing Data

### Task 0.1 - Data Loading

In [2]:
# Task 0 - Preprocessing Data

# Please add your address here as string
ADDRESS_TRAIN_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\train"
ADDRESS_TEST_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\test"

'''
# @author Pengnan Fan
# @param address: address of your train data file as string
# @return output: DataFrame{'comment':comments, 'isPositive':[0,1]}
def readTrainData(address):
    
    print("Start reading learning set")
    # Read negative data as DataFrame
    comment = []
    isPositive = []
    allNegFiles = glob.glob(address+"\\neg" + "/*.txt")
    count = 0
    lenN = len(allNegFiles)
    for file in allNegFiles:
        comment.append(pd.read_csv(file, quotechar=None, quoting=3, delimiter='\r').columns[0])
        isPositive.append(0)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenN)) * 100)
        sys.stdout.write("Start reading negative comments: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete reading negative comments")
    
    # Read positive data as DataFrame
    allPosFiles = glob.glob(address+"\\pos" + "/*.txt")
    count = 0
    lenP = len(allPosFiles)
    for file in allPosFiles:
        comment.append(pd.read_csv(file, quotechar=None, quoting=3, delimiter='\r').columns[0])
        isPositive.append(1)
        
        # This is for loading bar
        c = int((float(count) / float(lenP)) * 100)
        sys.stdout.write('\r')
        sys.stdout.write("Start reading positive comments: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete reading postive comments")
    print("Complete reading learning set\n")
    
    output = pd.DataFrame({'comment':comment, 'isPositive':isPositive})
    return output

# @author Pengnan Fan
# @param address: address of your test data file as string
# @return output: DataFrame{'comment':comments, 'isPositive':[0,1]}
# Note: all isPositive is initialized as 0
def readTestData(address):
    
    # Read negative data as DataFrame
    comment = []
    isPositive = []
    allFiles = glob.glob(address+"/*.txt")
    count = 0
    lenN = len(allFiles)
    for file in allFiles:
        comment.append(pd.read_csv(file, quotechar=None, quoting=3, delimiter='\r').columns[0])
        isPositive.append(0)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenN)) * 100)
        sys.stdout.write("Start reading test set: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete reading test set")
    
    output = pd.DataFrame({'comment':comment, 'isPositive':isPositive})
    return output

learningSet = readTrainData(ADDRESS_TRAIN_PENGNAN)
testSet = readTestData(ADDRESS_TEST_PENGNAN)
'''

# Advanced loader
# @return: Bunch
print("Start loading train set")
trainSet = sklearn.datasets.load_files(ADDRESS_TRAIN_PENGNAN)
print("Complete loading train set")

print("Start loading train set - negative")
trainSet_neg = sklearn.datasets.load_files(ADDRESS_TRAIN_PENGNAN, categories={"neg"})
print("Complete loading train set - negative")

print("Start loading train set - postive")
trainSet_pos = sklearn.datasets.load_files(ADDRESS_TRAIN_PENGNAN, categories={"pos"})
print("Complete loading train set - postive")

print("Start loading test set")
testSet = sklearn.datasets.load_files(ADDRESS_TEST_PENGNAN)
print("Complete loading test set")

Start loading train set
Complete loading train set
Start loading train set - negative
Complete loading train set - negative
Start loading train set - postive
Complete loading train set - postive
Start loading test set
Complete loading test set


In [3]:
# An example of the dataset
'''
print("The first learning set comment is:\n", learningSet['comment'][0],'\nisPositive: ', learningSet['isPositive'][0])
print("The total size of learning set: ", len(learningSet.comment))
print("The first test comment is:\n", testSet['comment'][0])
print("The total size of test set: ", len(testSet.comment))
'''
print("The data set has a structure of:")
for dataPoint in trainSet:
    print(dataPoint,": ", type(dataPoint))
print("Here are some examples:")
print("data: ", trainSet.data[0].decode("utf-8"))
print("target_names: ", trainSet.target_names[0])
print("The rest are irrelevent to this project.\nNOTE: Type \"data\" is in Byte mode. You need to transform them into string by using .decode(\"utf-8\")")

The data set has a structure of:
data :  <class 'str'>
filenames :  <class 'str'>
target_names :  <class 'str'>
target :  <class 'str'>
DESCR :  <class 'str'>
Here are some examples:
data:  Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.
target_names:  neg
The rest are irrelevent to

### Task 0.2 - Naive Word Frequency

In [4]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: naive word frequency
def wordsFrequencyNaive(dataSet):
    totalComments = []
    count = 0
    lenW = len(dataSet.data)
    for comment in dataSet.data:
        totalComments+=comment.decode("utf-8").lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("Start counting naive word frequency: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print("\nComplete counting naive word frequency")
    
    naiveCount = Counter(x for x in totalComments)
    return naiveCount

naiveFrequency = wordsFrequencyNaive(trainSet)
naiveFrequency_neg = wordsFrequencyNaive(trainSet_neg)
naiveFrequency_pos = wordsFrequencyNaive(trainSet_pos)

Start counting naive word frequency: [=================== ] 99%
Complete counting naive word frequency
Start counting naive word frequency: [=================== ] 99%
Complete counting naive word frequency
Start counting naive word frequency: [=================== ] 99%
Complete counting naive word frequency


In [5]:
# An example of naive word frequency
print(naiveFrequency.most_common(160))
print("Size of naiveFrequency: ", len(naiveFrequency))
print("Example: the = ", naiveFrequency['the'])
print(naiveFrequency_neg.most_common(160))
print("Size of naiveFrequency_neg: ", len(naiveFrequency_neg))
print("Example: the = ", naiveFrequency_neg['the'])
print(naiveFrequency_pos.most_common(160))
print("Size of naiveFrequency_pos:", len(naiveFrequency_pos))
print("Example: the = ", naiveFrequency_pos['the'])

[('the', 322174), ('a', 159949), ('and', 158556), ('of', 144459), ('to', 133965), ('is', 104170), ('in', 90521), ('i', 70477), ('this', 69711), ('that', 66287), ('it', 65489), ('/><br', 50935), ('was', 47023), ('as', 45098), ('for', 42839), ('with', 42725), ('but', 39757), ('on', 31618), ('movie', 30883), ('his', 29058), ('are', 28742), ('not', 28591), ('film', 27774), ('you', 27558), ('have', 27343), ('he', 26175), ('be', 25690), ('at', 22730), ('one', 22477), ('by', 21974), ('an', 21239), ('they', 20623), ('from', 19933), ('all', 19738), ('who', 19407), ('like', 18779), ('so', 18095), ('just', 17308), ('or', 16765), ('has', 16570), ('her', 16538), ('about', 16486), ("it's", 15969), ('some', 15280), ('if', 15184), ('out', 14509), ('what', 14053), ('very', 13633), ('when', 13608), ('more', 13168), ('there', 13093), ('she', 12234), ('would', 12027), ('even', 12008), ('good', 11926), ('my', 11766), ('only', 11566), ('their', 11317), ('no', 11271), ('really', 11064), ('had', 11041), ('whi

### Task 0.3 - Word Frequency without Stopword

In [6]:
# @author Pengnan Fan
# @acknowledgement Yuxiang Ma, for this function is edited based on his in miniproject1
# @param dataSet: set of comments
# @return naiveCount: word frequency without stopwords
def wordsFrequencyStopword(dataSet):
    totalString = str()
    count = 0
    lenW = len(dataSet.data)
    
    for comment in dataSet.data: 
        totalString = totalString + ' ' + comment.decode("utf-8").lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("Start counting naive word frequency: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    
    print("\nComplete counting word frequency without stopword")
    
    countDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    return countDict

stopwordFrequency = wordsFrequencyStopword(trainSet)
stopwordFrequency_neg = wordsFrequencyStopword(trainSet_neg)
stopwordFrequency_pos = wordsFrequencyStopword(trainSet_pos)

Start counting naive word frequency: [=================== ] 99%
Complete counting word frequency without stopword
Start counting naive word frequency: [=================== ] 99%
Complete counting word frequency without stopword
Start counting naive word frequency: [=================== ] 99%
Complete counting word frequency without stopword


In [7]:
# An example of word frequency without stopwords
print(stopwordFrequency.most_common(160))
print("Size of stopwordFrequency: ", len(stopwordFrequency))
print("Example: br = ", stopwordFrequency['br'])
print(stopwordFrequency_neg.most_common(160))
print("Size of stopwordFrequency_neg: ", len(stopwordFrequency_neg))
print("Example: movie = ", stopwordFrequency_neg['movie'])
print(stopwordFrequency_pos.most_common(160))
print("Size of stopwordFrequency_pos: ", len(stopwordFrequency_pos))
print("Example: movie = ", stopwordFrequency_pos['film'])

[('br', 101871), ('movie', 44047), ('film', 40159), ('one', 26795), ('like', 20281), ('good', 15147), ('time', 12727), ('even', 12655), ('would', 12436), ('story', 11988), ('really', 11738), ('see', 11479), ('well', 10667), ('much', 9764), ('get', 9311), ('bad', 9308), ('people', 9287), ('first', 9064), ('great', 9060), ('made', 8364), ('way', 8026), ('make', 8025), ('could', 7922), ('movies', 7668), ('think', 7297), ('characters', 7159), ('character', 7024), ('watch', 6974), ('two', 6906), ('films', 6889), ('seen', 6681), ('many', 6675), ('life', 6632), ('plot', 6589), ('acting', 6494), ('never', 6485), ('love', 6453), ('little', 6438), ('best', 6416), ('show', 6295), ('know', 6167), ('ever', 5992), ('better', 5740), ('still', 5624), ('say', 5396), ('scene', 5383), ('scenes', 5213), ('go', 5158), ('something', 5076), ('back', 4972), ('real', 4736), ('watching', 4606), ('though', 4566), ('old', 4526), ('thing', 4525), ('years', 4514), ('actors', 4488), ('director', 4449), ('work', 4374

### Task 0.4 - Number of Naive Existance of Words

In [9]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: list of words of num of naive existances
def numOfExistanceNaive(dataSet):
    totalComments = []
    count = 0
    lenW = len(dataSet.data)
    for comment in dataSet.data:
        commentSplit = comment.decode("utf-8").lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("Start counting number of naive existance: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print("\nComplete counting number of naive existance")
    
    naiveCount = Counter(x for x in totalComments)
    return naiveCount

naiveNumOfExistance = numOfExistanceNaive(trainSet)
naiveNumOfExistance_neg = numOfExistanceNaive(trainSet_neg)
naiveNumOfExistance_pos = numOfExistanceNaive(trainSet_pos)

Start counting number of naive existance: [=================== ] 99%
Complete counting number of naive existance
Start counting number of naive existance: [=================== ] 99%
Complete counting number of naive existance
Start counting number of naive existance: [=================== ] 99%
Complete counting number of naive existance


In [10]:
# An example of number of naive existance of words
print(naiveNumOfExistance.most_common(160))
print("Size of naiveNumOfExistance: ", len(naiveNumOfExistance))
print("Example: the = ", naiveNumOfExistance['the'])
print(naiveNumOfExistance_neg.most_common(160))
print("Size of naiveNumOfExistance_neg: ", len(naiveNumOfExistance_neg))
print("Example: the = ", naiveNumOfExistance_neg['the'])
print(naiveNumOfExistance_pos.most_common(160))
print("Size of naiveNumOfExistance_pos: ", len(naiveNumOfExistance_pos))
print("Example: the = ", naiveNumOfExistance_pos['the'])

[('the', 24771), ('a', 24143), ('and', 24072), ('of', 23705), ('to', 23440), ('is', 22312), ('this', 22248), ('in', 21885), ('it', 20248), ('that', 19723), ('i', 18615), ('for', 17650), ('but', 17354), ('with', 17231), ('was', 16042), ('as', 15859), ('on', 15141), ('/><br', 14665), ('not', 14361), ('have', 14079), ('be', 13777), ('are', 13744), ('movie', 13306), ('one', 12809), ('at', 12717), ('you', 12408), ('an', 12100), ('film', 11838), ('all', 11732), ('by', 11546), ('from', 11508), ('like', 11102), ('so', 10833), ('who', 10728), ('his', 10630), ('just', 10376), ('they', 10204), ('about', 10164), ('if', 9980), ('has', 9969), ('or', 9869), ('he', 9711), ('out', 9536), ('some', 9442), ('what', 9057), ('when', 8801), ("it's", 8785), ('there', 8600), ('very', 8551), ('more', 8551), ('only', 8184), ('good', 8171), ('even', 8114), ('would', 7944), ('my', 7728), ('can', 7673), ('see', 7517), ('no', 7510), ('up', 7494), ('really', 7358), ('had', 7332), ('than', 7062), ('which', 7030), ('th

### Task 0.5 - Number of Existance of Words without Stopwords

In [16]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: list of num of existances of words without stopwords
def numOfExistanceStopword(dataSet):
    totalString = str()
    count = 0
    lenW = len(dataSet.data)
    
    for comment in dataSet.data: 
        commentSplit = comment.decode("utf-8").lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        for word in wordsToAdd:
            totalString = totalString + ' ' + word
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("Start counting number of existance without stopwords: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    
    print("\nComplete counting number of naive existance without stopwords")
    
    countDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    return countDict

stopwordNumOfExistance = numOfExistanceStopword(trainSet)
stopwordNumOfExistance_neg = numOfExistanceStopword(trainSet_neg)
stopwordNumOfExistance_pos = numOfExistanceStopword(trainSet_pos)

Start counting number of existance without stopwords: [===                 ] 17%

KeyboardInterrupt: 

In [ ]:
# An example of number of existance of words without stopwords
print(stopwordNumOfExistance.most_common(160))
print("Size of stopwordNumOfExistance: ", len(stopwordNumOfExistance))
print("Example: br = ", stopwordNumOfExistance['br'])
print(naiveNumOfExistance_neg.most_common(160))
print("Size of stopwordNumOfExistance_neg: ", len(stopwordNumOfExistance_neg))
print("Example: movie = ", stopwordNumOfExistance_neg['movie'])
print(naiveNumOfExistance_pos.most_common(160))
print("Size of stopwordNumOfExistance_pos: ", len(stopwordNumOfExistance_pos))
print("Example: movie = ", stopwordNumOfExistance_pos['film'])

## Task 1 - Bernoulli Naive Bayes

In [ ]:
# @author Pengnan Fan
# @param dataSet: set for prediction
# @param totalWordFreq: word frequency of both neg and pos set
# @param negWordFreq: word frequency of neg set
# @param posWordFreq: word frequency of pos set
# @param numOfExamples: list {0 -> all, 1 -> pos, 2 -> neg}
# @return prediction
def bernoulliNaiveBayes(dataSet, totalWordFreq, negWordFreq, posWordFreq, numOfExamples):
    for comment in dataSet.data:
        word = comment.decode("utf-8").lower().split
        existanceAndPos = 1
        existanceAndNeg = 1
        for w in word:
            existanceAndPos+=posWordFreq[word]
            existanceAndNeg+=negWordFreq[word]
        probPos = (existanceAndPos+1)/(numOfExamples[1]+2)
        probNeg = (existanceAndNeg+1)/(numOfExamples[2]+2)
        